In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from funkagent import agents
from dotenv import load_dotenv
import asyncio
import os

In [3]:
# Load config file
load_dotenv()

True

## Define simple agent

In [4]:
# create a simple function for testing with
def circumference_calculator(radius: float, something: float = 4.4) -> float:
    """Calculates the circumference of a circle given the radius.

    :param radius: The radius of the circle as a single, positive number (float) - no math calculations (*,+,-,/) as input allowed.
    :return: The circumference of the circle
    """
    return 2 * 3.14 * radius

In [5]:
my_agent = agents.Agent(
    functions=[circumference_calculator]
)

await my_agent.ask("What is the circumference of a circle with radius 5 in one sentence?")

..

'The circumference of a circle with a radius of 5 is approximately 31.4 units.'

In [6]:
questions = [
    "What is love in one sentence?",
    "What is the circumference of a circle with radius 7 in one sentence?",
    "Who is Messi in one sentence?",
    "What is the circumference of a circle with radius 9 in one sentence?",
]

my_agents = [agents.Agent(functions=[circumference_calculator]) for _ in range(len(questions))]

results = await asyncio.gather(*[agent.ask(question) for question, agent in zip(questions, my_agents)])

for answer, question in zip(results, questions):
    print("\n")
    print(question)
    print("->")
    print(answer)

......

What is love in one sentence?
->
Love is a complex and multifaceted emotion that encompasses a deep affection, care, and connection between individuals, often characterized by selflessness and a strong desire for the well-being and happiness of the other person.


What is the circumference of a circle with radius 7 in one sentence?
->
The circumference of a circle with a radius of 7 inches is approximately 43.96 inches.


Who is Messi in one sentence?
->
Lionel Messi is an Argentine professional soccer player widely regarded as one of the greatest footballers of all time.


What is the circumference of a circle with radius 9 in one sentence?
->
The circumference of a circle with a radius of 9 is 56.52 units.


## Add cognitive search as vector store

In [7]:
# Load customized retriever _> filtering possible:
# await retriever.aget_relevant_documents_filter(query="Test 3 source1", **{"filter": "source eq 'A'"})
from retriever.AzureCognitiveSearchRetriever import AzureCognitiveSearchRetrieverWithFilter


In [8]:
retriever = AzureCognitiveSearchRetrieverWithFilter(content_key="text", top_k=3)

In [9]:
results = await retriever.aget_relevant_documents(query="Akademikeranteil")

print(" ".join([res.page_content for res in results]))

ALLE – PRODUKTE, PROZESSE, LÖSUNGEN

IVU.integration

Vernetzte Systeme
Wir stellen sicher, dass 
Ihre Systeme optimal 
zusammenarbeiten und 
alle Daten reibungslos 
fließen.

IVU.xpress

Effizienter Projektprozess
Unser Implementierungs-
prozess macht Ihr Projekt 
besser planbar und sorgt 
für einen schnellen Go-
Live.

IVU.support

Kompetenter Support
Auch nach der Inbetrieb-
nahme stehen Ihnen 
unsere kompetenten 
Ansprechpartner 
jederzeit zur Verfügung.

IVU.cloud

Verlässliches Hosting
Performant, hoch-
verfügbar, sicher: Wir 
übernehmen Hosting und 
Betrieb Ihres Systems in 
unserer Cloud.

  IVU.suite
   Integrierte IT-Lösungen für alle Aufgabenbereiche eines Verkehrsbetriebs: 

   von der Planung bis hin zur Abrechnung von Verkehrsverträgen.

IVU.consult

Strategische Beratung
Wir unterstützen Sie dabei, 
Ihr Angebot zu verbessern 
und Ihre Prozesse mit den 
Produkten der IVU.suite zu 
optimieren.

pageimage6.jpg





IVU TRAFFIC TECHNOLOGIES AG
KLUGE KÖPFE FÜR KOMPLEXE SYSTEM

In [10]:
async def get_IVU_context(input_to_search_for_context: str):
    """Get context from IVU database, very helpful to answer questions about IVU.

    :param: input_to_search_for_context: Text (word or a whole sentence) to get the relevant context from the IVU database through vector search.
    :return: IVU context that helps to answer questions about IVU.
    """
    results = await retriever.aget_relevant_documents(query=input_to_search_for_context)
    return " ".join([res.page_content for res in results])
    

In [11]:
my_agent = agents.Agent(
    functions=[get_IVU_context]
)

await my_agent.ask("Wie hoch ist der Akademikeranteil der IVU?")

..

'Es gibt keine spezifischen Informationen über den Akademikeranteil der IVU (IVU Traffic Technologies AG). In den verfügbaren Informationen wird erwähnt, dass die IVU nach "qualifizierten" Mitarbeitern sucht, wobei der Akademikeranteil über 80% liegt.\n\nEs ist jedoch wichtig zu beachten, dass dies nur allgemeine Informationen sind und keine detaillierten Zahlen bereitgestellt werden. Um genaue Informationen über den Akademikeranteil der IVU zu erhalten, empfehle ich, sich direkt an das Unternehmen zu wenden oder weitere aktuelle Informationen zu recherchieren.'

In [12]:
my_agent.chat_history

[{'role': 'system',
  'content': 'Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussion on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\nOverall, Assistant is a powerful system 

In [13]:
my_agent.internal_thoughts

[{'role': 'assistant',
  'function_call': <OpenAIObject at 0x155198ef0> JSON: {
    "name": "get_IVU_context",
    "arguments": "{\n  \"input_to_search_for_context\": \"Akademikeranteil\"\n}"
  },
  'content': None},
 {'role': 'function',
  'name': 'get_IVU_context',
  'content': 'ALLE – PRODUKTE, PROZESSE, LÖSUNGEN\n\nIVU.integration\n\nVernetzte Systeme\nWir stellen sicher, dass \nIhre Systeme optimal \nzusammenarbeiten und \nalle Daten reibungslos \nfließen.\n\nIVU.xpress\n\nEffizienter Projektprozess\nUnser Implementierungs-\nprozess macht Ihr Projekt \nbesser planbar und sorgt \nfür einen schnellen Go-\nLive.\n\nIVU.support\n\nKompetenter Support\nAuch nach der Inbetrieb-\nnahme stehen Ihnen \nunsere kompetenten \nAnsprechpartner \njederzeit zur Verfügung.\n\nIVU.cloud\n\nVerlässliches Hosting\nPerformant, hoch-\nverfügbar, sicher: Wir \nübernehmen Hosting und \nBetrieb Ihres Systems in \nunserer Cloud.\n\n  IVU.suite\n   Integrierte IT-Lösungen für alle Aufgabenbereiche eines Ver